In [ ]:
# 导入必要的库
import cv2 as cv
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from grayscale import *
from binarization import *
from sharpening import *


In [ ]:
# 图像显示函数
def show_image(image, title='Image'):
    plt.imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis('off')
    plt.show()

# 图像处理函数
def process_image(image, method, sub_option, kernel_size=None, sigmaX=None, d=None, sigmaColor=None, sigmaSpace=None, threshold=None):

    gray_image = grayscale(image)
    processed_image = None
    
    if method == '灰度化':
        if sub_option == '灰度化':
            processed_image = gray_image
        elif sub_option == '灰度直方图修正':
            processed_image = histogram_equalization(gray_image)
        elif sub_option == '线性变换':
            processed_image = linear_transformation(gray_image)
        elif sub_option == '对数变换':
            processed_image = log_transformation(gray_image)
        elif sub_option == '指数变换':
            processed_image = exponential_transformation(gray_image)
        plot_histogram(processed_image) # 绘制直方图
    elif method == '二值化':
        if sub_option == '固定阈值分割':
            processed_image = fixed_threshold(gray_image, threshold)
        elif sub_option == '自适应均值阈值分割':
            processed_image = adaptive_mean_threshold(gray_image)
        elif sub_option == '自适应高斯阈值分割':
            processed_image = adaptive_gaussian_threshold(gray_image)
        elif sub_option == 'Otsu自动计算阈值分割':
            processed_image = otsu_threshold(gray_image)
        
    elif method == '滤波':
        if sub_option == '均值滤波':
            kernel_size = kernel_size or 5
            processed_image = cv.blur(gray_image, (kernel_size, kernel_size))
        elif sub_option == '高斯滤波':
            kernel_size = kernel_size or 5
            sigmaX = sigmaX or 1.5
            processed_image = cv.GaussianBlur(gray_image, (kernel_size, kernel_size), sigmaX)
        elif sub_option == '中值滤波':
            kernel_size = kernel_size or 5
            processed_image = cv.medianBlur(gray_image, kernel_size)
        elif sub_option == '双边滤波':
            d = d or 9
            sigmaColor = sigmaColor or 75
            sigmaSpace = sigmaSpace or 75
            processed_image = cv.bilateralFilter(gray_image, d, sigmaColor, sigmaSpace)

    elif method == '锐化': 
        if sub_option == 'Sobel算子':
            processed_image = sobel_sharpen(image)
        elif sub_option == 'Laplacian算子':
            processed_image = laplacian_sharpen(image)
    return processed_image

In [ ]:
# 文件上传控件
file_upload = widgets.FileUpload(
    accept='image/*',
    multiple=False
)

# 方法选择下拉菜单
method_dropdown = widgets.Dropdown(
    options=['灰度化', '二值化', '滤波', '锐化'],  
    value='灰度化',
    description='处理方法:'
)

# 子选项选择下拉菜单
sub_option_dropdown = widgets.Dropdown(
    options=['灰度化', '灰度直方图修正', '线性变换', '对数变换', '指数变换'],
    value='灰度化',
    description='子选项:'
)

# 阈值滑动条，默认隐藏
threshold_slider = widgets.IntSlider(
    min=0,
    max=255,
    step=1,
    value=127,
    description='阈值:',
    layout={'visibility': 'hidden'}
)

# 滤波器参数控件，默认隐藏
filter_params = widgets.HBox([
    widgets.IntSlider(min=1, max=15, step=2, value=5, description='Kernel Size:', layout={'visibility': 'hidden'}),
    widgets.FloatSlider(min=0.1, max=5.0, step=0.1, value=1.5, description='Sigma X:', layout={'visibility': 'hidden'}),
    widgets.IntSlider(min=1, max=15, step=2, value=9, description='Diameter:', layout={'visibility': 'hidden'}),
    widgets.IntSlider(min=1, max=100, step=1, value=75, description='Sigma Color:', layout={'visibility': 'hidden'}),
    widgets.IntSlider(min=1, max=100, step=1, value=75, description='Sigma Space:', layout={'visibility': 'hidden'})
])

# 处理按钮
process_button = widgets.Button(
    description='处理',
)

# 输出区域
output = widgets.Output()

# 文件上传事件处理
def on_file_upload_changed(change):
    global image
    uploaded_files = change['new']
    if isinstance(uploaded_files, tuple):
        uploaded_files = {file.name: {'content': file.content} for file in uploaded_files}
    
    for filename, file_info in uploaded_files.items():
        content = file_info['content']
        image = cv.imdecode(np.frombuffer(content, np.uint8), -1)
        with output:
            clear_output(wait=True)
            show_image(image, '原始图像')

file_upload.observe(on_file_upload_changed, names='value')

# 处理按钮点击事件处理
def on_process_button_clicked(b):
    with output:
        clear_output(wait=True)
        if method_dropdown.value == '二值化' and sub_option_dropdown.value == '固定阈值分割':
            processed_image = process_image(image, method_dropdown.value, sub_option_dropdown.value, threshold=threshold_slider.value)
        elif method_dropdown.value == '滤波':
            kernel_size = filter_params.children[0].value
            sigmaX = filter_params.children[1].value
            d = filter_params.children[2].value
            sigmaColor = filter_params.children[3].value
            sigmaSpace = filter_params.children[4].value
            processed_image = process_image(image, method_dropdown.value, sub_option_dropdown.value, kernel_size, sigmaX, d, sigmaColor, sigmaSpace)
        else:
            processed_image = process_image(image, method_dropdown.value, sub_option_dropdown.value)
        
        fig, axs = plt.subplots(1, 2, figsize=(12, 6))
        axs[0].imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))
        axs[0].set_title('原始图像')
        axs[0].axis('off')
        
        axs[1].imshow(cv.cvtColor(processed_image, cv.COLOR_BGR2RGB) if len(processed_image.shape) == 3 else processed_image, cmap='gray')
        axs[1].set_title('处理后的图像')
        axs[1].axis('off')
        
        plt.show()

process_button.on_click(on_process_button_clicked)

# 方法改变事件处理
def on_method_change(change):
    if change['new'] == '灰度化':
        sub_option_dropdown.options = ['灰度化', '灰度直方图修正', '线性变换', '对数变换', '指数变换']
        threshold_slider.layout.visibility = 'hidden'
        filter_params.layout.visibility = 'hidden'
    elif change['new'] == '二值化':
        sub_option_dropdown.options = ['固定阈值分割', '自适应均值阈值分割', '自适应高斯阈值分割', 'Otsu自动计算阈值分割']
        if sub_option_dropdown.value == '固定阈值分割':
            threshold_slider.layout.visibility = 'visible'
        else:
            threshold_slider.layout.visibility = 'hidden'
        filter_params.layout.visibility = 'hidden'
    elif change['new'] == '滤波':
        sub_option_dropdown.options = ['均值滤波', '高斯滤波', '中值滤波', '双边滤波']
        filter_params.layout.visibility = 'visible'
        if sub_option_dropdown.value in ['均值滤波', '高斯滤波']:
            filter_params.children[0].layout.visibility = 'visible'  # Kernel Size
            filter_params.children[1].layout.visibility = 'visible' if sub_option_dropdown.value == '高斯滤波' else 'hidden'  # Sigma X
            filter_params.children[2].layout.visibility = 'hidden'  # Diameter
            filter_params.children[3].layout.visibility = 'hidden'  # Sigma Color
            filter_params.children[4].layout.visibility = 'hidden'  # Sigma Space
        elif sub_option_dropdown.value == '双边滤波':
            filter_params.children[0].layout.visibility = 'hidden'  # Kernel Size
            filter_params.children[1].layout.visibility = 'hidden'  # Sigma X
            filter_params.children[2].layout.visibility = 'visible'  # Diameter
            filter_params.children[3].layout.visibility = 'visible'  # Sigma Color
            filter_params.children[4].layout.visibility = 'visible'  # Sigma Space
        else:
            filter_params.children[0].layout.visibility = 'visible'  # Kernel Size
            filter_params.children[1].layout.visibility = 'hidden'  # Sigma X
            filter_params.children[2].layout.visibility = 'hidden'  # Diameter
            filter_params.children[3].layout.visibility = 'hidden'  # Sigma Color
            filter_params.children[4].layout.visibility = 'hidden'  # Sigma Space
        
    elif change['new'] == '锐化':
        sub_option_dropdown.options = ['Sobel算子', 'Laplacian算子']
        threshold_slider.layout.visibility = 'hidden'
        filter_params.layout.visibility = 'hidden'

method_dropdown.observe(on_method_change, names='value')

# 子选项改变事件处理
def on_sub_option_change(change):
    if method_dropdown.value == '二值化' and change['new'] == '固定阈值分割':
        threshold_slider.layout.visibility = 'visible'
    else:
        threshold_slider.layout.visibility = 'hidden'
    
    if method_dropdown.value == '滤波':
        if change['new'] in ['均值滤波', '高斯滤波']:
            filter_params.children[0].layout.visibility = 'visible'  # Kernel Size
            filter_params.children[1].layout.visibility = 'visible' if change['new'] == '高斯滤波' else 'hidden'  # Sigma X
            filter_params.children[2].layout.visibility = 'hidden'  # Diameter
            filter_params.children[3].layout.visibility = 'hidden'  # Sigma Color
            filter_params.children[4].layout.visibility = 'hidden'  # Sigma Space
        elif change['new'] == '双边滤波':
            filter_params.children[0].layout.visibility = 'hidden'  # Kernel Size
            filter_params.children[1].layout.visibility = 'hidden'  # Sigma X
            filter_params.children[2].layout.visibility = 'visible'  # Diameter
            filter_params.children[3].layout.visibility = 'visible'  # Sigma Color
            filter_params.children[4].layout.visibility = 'visible'  # Sigma Space
        else:
            filter_params.children[0].layout.visibility = 'visible'  # Kernel Size
            filter_params.children[1].layout.visibility = 'hidden'  # Sigma X
            filter_params.children[2].layout.visibility = 'hidden'  # Diameter
            filter_params.children[3].layout.visibility = 'hidden'  # Sigma Color
            filter_params.children[4].layout.visibility = 'hidden'  # Sigma Space

sub_option_dropdown.observe(on_sub_option_change, names='value')

# 显示控件
display(file_upload, method_dropdown, sub_option_dropdown, threshold_slider, filter_params, process_button, output)